In [79]:
import pickle   # importing pickle for saving and loading machine learning models
import pandas as pd  # importing pandas for analyzing, cleaning, exploring, and manipulating data
from sklearn.model_selection import train_test_split  # importing train_test_split for spliting the data
from preprocess1 import *  # importing * for import all functions at once
from imblearn.over_sampling import SMOTE  # importing SMOTE for Balancing the Data

In [80]:
with open('svm.pkl','rb')as f:
    model=pickle.load(f)


In [81]:
model

ColumnTransformer(transformers=[('OHE columns', OneHotEncoder(),
                                 ['Self_Employed', 'Property_Area', 'Gender']),
                                ('Label_encoder', ModifiedLabelEncoder(),
                                 'Married'),
                                ('standard_scaler', StandardScaler(),
                                 ['ApplicantIncome', 'CoapplicantIncome',
                                  'LoanAmount']),
                                ('custom',
                                 FunctionTransformer(func=<function divide_by_12 at 0x00000280A0E95B20>),
                                 ['Loan_Amount_Term']),
                                ('pass through',
                                 FunctionTransformer(func=<function same at 0x00000280A2F156C0>),
                                 ['Credit_History']),
                                ('ordinal dependents',
                                 OrdinalEncoder(categories=[['0', '1', '2',
                                                             '3+']]),
                                 ['Dependents']),
                                ('ordinal education',
                                 OrdinalEncoder(categories=[['Not Graduate',
                                                             'Graduate']]),
                                 ['Education'])])

In [86]:
df=pd.read_csv("loan_approved.csv")  

In [87]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status (Approved)
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [88]:
df.loc[df['Gender'].isnull(),'Gender']="Male"
df.loc[df['Married'].isnull(),'Married']="Yes"
df.loc[df['Dependents'].isnull(),'Dependents']=='0'
df.loc[df['Self_Employed'].isnull(),'Self_Employed']='No'
df.loc[df['LoanAmount'].isnull(),'LoanAmount']=df['LoanAmount'].median()
df.loc[df['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=360.0
df.loc[df['Credit_History'].isnull(),'Credit_History']=0.0 

In [95]:
df.Dependents.unique()

array(['0', '1', '2', '3+', nan], dtype=object)

In [105]:
df['Dependents']=df['Dependents'].replace('nan','0').fillna('0')

In [107]:
df.Dependents.unique()

array(['0', '1', '2', '3+'], dtype=object)

In [108]:
df.drop(['Loan_ID'],axis=1,inplace=True)

In [109]:
df.isnull().sum()

Gender                    0
Married                   0
Dependents                0
Education                 0
Self_Employed             0
ApplicantIncome           0
CoapplicantIncome         0
LoanAmount                0
Loan_Amount_Term          0
Credit_History            0
Property_Area             0
Loan_Status (Approved)    0
dtype: int64

In [110]:
x=df.drop(['Loan_Status (Approved)'],axis=1)
y=df['Loan_Status (Approved)']

In [111]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)


In [112]:
x_train.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [113]:
x_train

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
92,Male,Yes,2,Not Graduate,No,3273,1820.0,81.0,360.0,1.0,Urban
304,Male,No,0,Graduate,No,4000,2500.0,140.0,360.0,1.0,Rural
68,Male,Yes,3+,Not Graduate,Yes,7100,0.0,125.0,60.0,1.0,Urban
15,Male,No,0,Graduate,No,4950,0.0,125.0,360.0,1.0,Urban
211,Male,Yes,3+,Graduate,No,3430,1250.0,128.0,360.0,0.0,Semiurban
...,...,...,...,...,...,...,...,...,...,...,...
71,Male,Yes,2,Not Graduate,Yes,1875,1875.0,97.0,360.0,1.0,Semiurban
106,Male,Yes,2,Graduate,No,11417,1126.0,225.0,360.0,1.0,Urban
270,Female,No,0,Graduate,No,3237,0.0,30.0,360.0,1.0,Urban
435,Female,Yes,0,Graduate,No,10047,0.0,128.0,240.0,1.0,Semiurban


In [54]:
x_train.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [115]:
model.fit_transform(x_train)

array([[1., 0., 0., ..., 1., 2., 0.],
       [1., 0., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 3., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.]])

In [116]:
x_train

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
92,Male,Yes,2,Not Graduate,No,3273,1820.0,81.0,360.0,1.0,Urban
304,Male,No,0,Graduate,No,4000,2500.0,140.0,360.0,1.0,Rural
68,Male,Yes,3+,Not Graduate,Yes,7100,0.0,125.0,60.0,1.0,Urban
15,Male,No,0,Graduate,No,4950,0.0,125.0,360.0,1.0,Urban
211,Male,Yes,3+,Graduate,No,3430,1250.0,128.0,360.0,0.0,Semiurban
...,...,...,...,...,...,...,...,...,...,...,...
71,Male,Yes,2,Not Graduate,Yes,1875,1875.0,97.0,360.0,1.0,Semiurban
106,Male,Yes,2,Graduate,No,11417,1126.0,225.0,360.0,1.0,Urban
270,Female,No,0,Graduate,No,3237,0.0,30.0,360.0,1.0,Urban
435,Female,Yes,0,Graduate,No,10047,0.0,128.0,240.0,1.0,Semiurban


In [117]:
x_train_transform=model.transform(x_train)

In [118]:
x_test_transform=model.transform(x_test)

In [119]:
len(y_train)

460

In [120]:
len(x_train)

460

In [121]:
y_test

350    Y
377    Y
163    Y
609    Y
132    Y
      ..
535    Y
468    Y
582    Y
291    N
75     N
Name: Loan_Status (Approved), Length: 154, dtype: object

In [144]:
y_test=y_test.replace({'Y':1,'N':0})
y_test

350    1
377    1
163    1
609    1
132    1
      ..
535    1
468    1
582    1
291    0
75     0
Name: Loan_Status (Approved), Length: 154, dtype: int64

In [138]:
y_train=y_train.replace({'Y':1,'N':0})
y_train

92     1
304    1
68     1
15     1
211    0
      ..
71     1
106    1
270    1
435    1
102    1
Name: Loan_Status (Approved), Length: 460, dtype: int64

In [129]:
x_train

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
92,Male,Yes,2,Not Graduate,No,3273,1820.0,81.0,360.0,1.0,Urban
304,Male,No,0,Graduate,No,4000,2500.0,140.0,360.0,1.0,Rural
68,Male,Yes,3+,Not Graduate,Yes,7100,0.0,125.0,60.0,1.0,Urban
15,Male,No,0,Graduate,No,4950,0.0,125.0,360.0,1.0,Urban
211,Male,Yes,3+,Graduate,No,3430,1250.0,128.0,360.0,0.0,Semiurban
...,...,...,...,...,...,...,...,...,...,...,...
71,Male,Yes,2,Not Graduate,Yes,1875,1875.0,97.0,360.0,1.0,Semiurban
106,Male,Yes,2,Graduate,No,11417,1126.0,225.0,360.0,1.0,Urban
270,Female,No,0,Graduate,No,3237,0.0,30.0,360.0,1.0,Urban
435,Female,Yes,0,Graduate,No,10047,0.0,128.0,240.0,1.0,Semiurban


In [127]:
from imblearn.over_sampling import SMOTE

In [139]:
sm=SMOTE()
x_smote,y_smote=sm.fit_resample(x_train_transform,y_train)

In [140]:
len(y_train)

460

In [141]:
len(y_smote)

644

In [142]:
y_smote

0      1
1      1
2      1
3      1
4      0
      ..
639    0
640    0
641    0
642    0
643    0
Name: Loan_Status (Approved), Length: 644, dtype: int64

In [146]:
len(x_smote)

644

MODEL BUILDING

In [147]:
from sklearn.svm import SVC

In [148]:
svm=SVC()

In [149]:
svm.fit(x_smote,y_smote)

SVC()

In [150]:
y_pred=svm.predict(x_test_transform)

In [151]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [152]:
from sklearn.metrics import classification_report,accuracy_score,f1_score

In [155]:
report=classification_report(y_pred,y_test)
print(report)

              precision    recall  f1-score   support

           0       0.06      0.43      0.10         7
           1       0.96      0.65      0.78       147

    accuracy                           0.64       154
   macro avg       0.51      0.54      0.44       154
weighted avg       0.92      0.64      0.75       154



HYPER PARAMETER TUNING

In [156]:
from itertools import product

In [165]:
param_grid_linear={
    'C':[0.1,5,10,50,60,70],
    'kernel':['linear'],
    'gamma':['scale','auto']
}
param_grid_rbf={
    'C':[0.1,5,10,50,60,70],
    'kernel':['rbf'],
    'gamma':['auto','scale']
}
param_grid_poly={
    'C':[0.1,5,10,50,60,70],
    'kernel':['poly'],
    'gamma':['auto','scale'],
    'degree':[2,3,4]
}

In [166]:
from sklearn.model_selection import GridSearchCV

In [167]:
model=SVC()

In [168]:
grid=GridSearchCV(model,param_grid=param_grid_rbf,refit=True,verbose=1,scoring='f1',cv=3)

In [169]:
grid.fit(x_smote,y_smote)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 5, 10, 50, 60, 70],
                         'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
             scoring='f1', verbose=1)

In [170]:
print(grid.best_params_)

{'C': 70, 'gamma': 'auto', 'kernel': 'rbf'}


In [176]:
model1=SVC(C=70,gamma='auto',kernel='rbf')


In [177]:
model1.fit(x_smote,y_smote)

SVC(C=70, gamma='auto')

In [185]:
y_pr=grid.predict(x_test_transform)
y_pr

array([1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0])

In [183]:
from sklearn.metrics import classification_report

In [186]:
report=classification_report(y_test,y_pr)
print(report)

              precision    recall  f1-score   support

           0       0.59      0.48      0.53        54
           1       0.75      0.82      0.78       100

    accuracy                           0.70       154
   macro avg       0.67      0.65      0.66       154
weighted avg       0.69      0.70      0.69       154

